In [1]:
import nltk
import pandas as pd
import praw
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [2]:
reddit = praw.Reddit(client_id='m0T7JivawYEgBVsff5M08A',
                     client_secret='PQC4K_w0Fjavpu2g4acaK8DnMhXZQg',
                     user_agent='OrbAcct')

In [4]:
mods = {}
#pattern = r"[A-Za-z][A-Za-z]\d\d\d\d"
pattern = input("Input Mod: ")
nus = reddit.subreddit('nus')
for submission in nus.new(limit=200):
    search = re.search(pattern, submission.title)
    if search:
        mod = submission.title[search.start():search.end()]
        if mod not in mods.keys():
            mods[mod] = []
        mods[mod].append((submission.title, [comment.body for comment in submission.comments]))
print(mods)

Input Mod: CS2040C
{'CS2040C': [('CS2040C', ['Abdul bari on youtube saved my life', 'Visualgo helped for me.. Good luck for this mod!', 'No need prepare, can means can. /s', 'CLRS', 'You can use Steven halim website for cs2040c and visualgo.The website got a schedule if you want follow for a headstart', 'Start practicing kattis. You can use the previous sem questions as a guide', 'Steven Halim will take good care of you actually no need to prepare content wise. i think just practice some C++ and STL is enough alr', "I think it should be sufficient to watch the VisuAlgo content since that's whats used for lectures too. Also on Steven Halim's website there are links to the kattis questions which should serve as a baseline on the C++ ability needed.", 'Bjarne Stroustrup’s book 😁\n… this recommendation is only for the truly kiasu', 'No need to prepare in advance. Just follow Steven‘s pace'])]}


# Scraping

In [3]:
#Selenium, BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re

def scrape_n_posts(mod, n):
    PATH = "C:/Users/Yan Rong/Documents/programming/Orbital/testsite/chromedriver.exe"
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")

    option.add_experimental_option(
    "prefs", {"profile.default_content_setting_values.notifications": 1}
)
    driver = webdriver.Chrome(options = option, executable_path = PATH)
    driver.get("https://www.reddit.com/r/nus/")
    search = driver.find_element_by_id("header-search-bar")
    search.send_keys(mod)
    time.sleep(3)
    search.send_keys(Keys.RETURN)
    
    try:
        main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='QBfRw7Rj8UkxybFpX-USO']")))
    except:
        driver.close()
        return "Error"

    result = []
    scale = 0
    pattern = r"[A-Za-z][A-Za-z]\d\d\d\d"
    while scale < n:
        links = main.find_elements(By.XPATH,f'//a[contains(@href,"{mod.lower()}")]')
        links[scale * 2].click()
        driver.switch_to.window(driver.window_handles[1])
        url = driver.current_url
        driver.get(url)
        html =  BeautifulSoup(driver.page_source, "html.parser")
        comments = html.find_all("p",{"class":"_1qeIAgB0cPwnLhDF9XSiJM"})
        for comment in comments:
            result.append(comment.text)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        scale += 1
    driver.close()
    return result

In [7]:
CS1010 = scrape_n_posts("CS1010", 20)

In [9]:
CS1010

["Hi, I'm a y1 student taking cs1010 without a CS background, my own major and diploma is in healthcare. Since they added CS as a compulsory GE mod as of this semester, I had no choice but to take it. Out of curiosity how difficult is it to pass and get a D? I'm looking to s/u and just clear this CS mod. Currently I'm struggling to complete the weekly tutorial coding questions, as you must get an A grade to get the full percentage. The things they teach during lectures is so far removed from the actual practical questions in terms of difficulty. It's honestly quite ridiculous with regard to the learning curve of cs1010. How exactly will the tests be graded? If you don't complete the question, will marks not be given at all? I'm genuinely worried about just passing this mod.",
 'From what I understand taking cs1010e, they test on your logic so as long as you attempt with acceptable logic, even if you fail the test cases, you will get marks for the exams',
 "How'd you do tho? Were the ex

In [8]:
BT2102 = scrape_n_posts("BT2102", 2)
MA1521 = scrape_n_posts("MA1521", 2)
CS2030 = scrape_n_posts("CS2030", 2)
IS2101 = scrape_n_posts("IS2101", 2)

# Sentiment Analysis

In [16]:
#NLTK VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

def SIA_analyse_sent(comments):
    sia = SIA()
    results = []
    for comment in comments:
        pol_score = sia.polarity_scores(comment)
        pol_score["Comment"] = comment
        results.append(pol_score)

    df = pd.DataFrame.from_records(results)
    print(df["compound"].mean())
    return df

In [13]:
#TextBlob
from textblob import TextBlob

def TB_analyse_sent(comments):
    results = []
    for comment in comments:
        entry = {}
        tb = TextBlob(comment)
        #Subjectivity
        entry["Score"] = tb.polarity
        entry["Comment"] = comment
        results.append(entry)
        
    df = pd.DataFrame.from_records(results)
    print(df["Score"].mean())
    return df

In [17]:
SIA_analyse_sent(["The more messed up part is that different people have different question, which is not even fair at all"])

-0.5654


,neg,neu,pos,compound,Comment
0,0.215,0.785,0.0,-0.5654,The more messed up part is that different peop...


In [254]:
def MODeRATE(MOD, n):
    return SIA_analyse_sent(scrape_n_posts(MOD, n))

In [221]:
MODeRATE("CS1010S", 2)

0.23067105263157897


,neg,neu,pos,compound,Comment
0,0.000,1.000,0.000,0.0000,now that's what I call a bell curve...oh wait
1,0.000,1.000,0.000,0.0000,Bell curve turned into a Ln curve
2,0.000,1.000,0.000,0.0000,Exponential curve
3,0.134,0.594,0.272,0.4215,this one like exponential RV. my goodness i th...
4,0.440,0.560,0.000,-0.6705,The 0 bar looks high enough to die falling off.
5,0.000,0.588,0.412,0.4215,"Nice, I scored the mode score"
6,0.000,0.692,0.308,0.5994,as of this very moment the cs1010s chat is ril...
7,0.000,0.000,0.000,0.0000,
8,0.000,1.000,0.000,0.0000,btw median is 6.5
9,0.000,0.909,0.091,0.3804,on a side note im actually very curious how Pr...
